In [92]:
from ghapi.all import GhApi
import requests as r
import pandas as pd

### YOU MUST SET TOKEN https://github.com/settings/tokens
token = ""
ghapi = GhApi(token=token)

In [217]:
# Get all repos in org.
rsp = r.get("https://api.github.com/orgs/tapis-project/repos?per_page=100")
repo_dicts = rsp.json()

In [218]:
# Create dataframe
df = pd.DataFrame(columns=["repo", "title", "created_at"])

# Get all issues in repo
for repo in repo_dicts:
    # Printing which repo we're going through now
    print(f"Getting issues for: {repo['full_name']}\r", end="")
    
    # Request issues for repo with Github api
    rsp = r.get(f"https://api.github.com/repos/{repo['full_name']}/issues?per_page=100",
                headers={"Authorization": f"token {token}"})
    issue_dicts = rsp.json()
    
    # per_page=100 (max). If you need more pages, code has to be added to go through all pages.
    if len(issue_dicts) == 99:
        print("Github api only allows 100 results per page. We've reached that. Add code to deal with pagniation")
    #print(len(issue_dicts))
    
    # Create df row for each issue
    # Note, convert created_at to datetime
    for issue in issue_dicts:
        df = df.append({'repo': repo['full_name'],
                        'title': issue['title'],
                        'created_at': pd.to_datetime(issue['created_at'], format = '%Y-%m-%dT%H:%M:%S')},
                       ignore_index=True)
    
    # Debug, only run until first repo with issues.
    if issue_dicts and False:
        break
        
# Create DatetimeIndex for "created_at" col. Allows for betweens and other panda functions (loc for example)
df.set_index('created_at', inplace=True)

In [221]:
# Get values with created_at greater than X
newdf = df[(df.index > '2021-08-01')]

print(len(newdf))
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.expand_frame_repr', False):  # more options can be specified also
    print(newdf)

184
                                                           repo                                              title
created_at                                                                                                        
2022-07-08 15:12:29+00:00              tapis-project/tapis-java            Add commit hash to healthcheck response
2022-07-08 15:10:35+00:00              tapis-project/tapis-java                 Improve job resubmit error message
2022-07-08 15:07:28+00:00              tapis-project/tapis-java        Improve Jobs jobArgSpec usage documentation
2022-07-01 12:52:51+00:00              tapis-project/tapis-java                 Job Shared App Context readthedocs
2022-06-23 13:55:41+00:00              tapis-project/tapis-java     Stop unnecessary role generation at SK startup
2022-06-21 20:46:46+00:00              tapis-project/tapis-java                    Jobs Shared App Context support
2022-06-17 14:41:01+00:00              tapis-project/tapis-java             